In [13]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt

### Load GTD

In [14]:
FILE = '../globalterrorismdb_0718dist.csv'
df = pd.read_csv(FILE, index_col='eventid', encoding='ISO-8859-1')

df.head(2)

C:\Users\Abe-r\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
eventid,,,,,,,,,,,,,,,,,,,,,
197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,North America,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN


### Minimized GTD

In [15]:
minified_cols = ['iyear', 'imonth', 'iday', 'country_txt', 'extended', 
                'country', 'region', 'success', 'attacktype1', 'targtype1',
               'natlty1', 'weaptype1']

gtd = df[minified_cols]

print("number of countries", len(gtd.country_txt.unique()))
gtd.head(5)

gtd.country_txt.unique()

number of countries 205


array(['Dominican Republic', 'Mexico', 'Philippines', 'Greece', 'Japan',
       'United States', 'Uruguay', 'Italy', 'East Germany (GDR)',
       'Ethiopia', 'Guatemala', 'Venezuela', 'West Germany (FRG)',
       'Switzerland', 'Jordan', 'Spain', 'Brazil', 'Egypt', 'Argentina',
       'Lebanon', 'Ireland', 'Turkey', 'Paraguay', 'Iran',
       'United Kingdom', 'Colombia', 'Bolivia', 'Nicaragua',
       'Netherlands', 'Belgium', 'Canada', 'Australia', 'Pakistan',
       'Zambia', 'Sweden', 'Costa Rica', 'South Yemen', 'Cambodia',
       'Israel', 'Poland', 'Taiwan', 'Panama', 'Kuwait',
       'West Bank and Gaza Strip', 'Austria', 'Czechoslovakia', 'India',
       'France', 'South Vietnam', 'Brunei', 'Zaire',
       "People's Republic of the Congo", 'Portugal', 'Algeria',
       'El Salvador', 'Thailand', 'Haiti', 'Sudan', 'Morocco', 'Cyprus',
       'Myanmar', 'Afghanistan', 'Peru', 'Chile', 'Honduras',
       'Yugoslavia', 'Ecuador', 'New Zealand', 'Malaysia', 'Singapore',
       'Bot

In [18]:
# convert East and West Germany to Germany only
gtd.at[gtd[gtd['country_txt'] == 'East Germany (GDR)'].index, 'country_txt'] = 'Germany'
gtd.at[gtd[gtd['country_txt'] == 'West Germany (FRG)'].index, 'country_txt'] = 'Germany'

gtd[gtd['country_txt'] == 'Germany'].head(5)

C:\Users\Abe-r\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,iyear,imonth,iday,country_txt,extended,country,region,success,attacktype1,targtype1,natlty1,weaptype1
eventid,,,,,,,,,,,,
197001100001,1970,1,10,Germany,0,499,9,1,3,2,75.0,6
197001280001,1970,1,28,Germany,0,499,9,1,7,3,75.0,8
197002100001,1970,2,10,Germany,0,362,8,1,2,6,97.0,5
197002130002,1970,2,13,Germany,0,362,8,1,7,14,75.0,8
197002210001,1970,2,21,Germany,0,362,8,1,3,6,15.0,6


### Combine Economic Data (until 2016 only) with GTD
- GDP
- Inflation
- Population

In [19]:
gdp = pd.read_csv("datahub/gdp_csv.csv")
inf = pd.read_csv("datahub/inflation-gdp_csv.csv")
pop = pd.read_csv("datahub/population_csv.csv")

gtd['gdp'] = 0
gtd['inf'] = 0
gtd['pop'] = 0


for i in gtd.country_txt.unique():
    print(i)
    for y in gtd[gtd['country_txt'] == i].iyear.unique():
        print(y, " ", end='')
        
        gdp_ = 0 if not gdp[(gdp['Country Name'] == i) & (gdp['Year'] == y)]['Value'].tolist() else gdp[(gdp['Country Name'] == i) & (gdp['Year'] == y)]['Value'].tolist()[0]
        inf_ = 0 if not inf[(inf['Country'] == i) & (inf['Year'] == y)]['Inflation'].tolist() else inf[(inf['Country'] == i) & (inf['Year'] == y)]['Inflation'].tolist()[0]
        pop_ = 0 if not pop[(pop['Country Name'] == i) & (pop['Year'] == y)]['Value'].tolist() else pop[(pop['Country Name'] == i) & (pop['Year'] == y)]['Value'].tolist()[0]

        gtd.at[gtd[(gtd['country_txt'] == i) & (gtd['iyear'] == y)].index, 'gdp'] = gdp_
        gtd.at[gtd[(gtd['country_txt'] == i) & (gtd['iyear'] == y)].index, 'inf'] = inf_
        gtd.at[gtd[(gtd['country_txt'] == i) & (gtd['iyear'] == y)].index, 'pop'] = pop_
        print()
    print()
    
    
#print(gdp[(gdp['Country Name'] == 'Philippines') & (gdp['Year'] == year['iyear'])])
# gdp[(gdp['Country Name'] == 'Philippines') & (gdp['Year'] == 1960)]
# gtd[gtd['country_txt'] == 'Philippines']

"""
gtd[gtd['country_txt'] == 'Philippines'].index

gtd.at[gtd[(gtd['country_txt'] == 'Philippines') & (gtd['iyear'] == 1970)].index, 'gdp'] = gdp[(gdp['Country Name'] == 'Philippines') & (gdp['Year'] == 1970)]['Value'].tolist()[0]

gtd[gtd['country_txt'] == 'Philippines']
"""

# gtd[(gtd['country_txt'] == 'Philippines') & (gtd['iyear'] == 2016)]

C:\Users\Abe-r\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Abe-r\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Abe-r\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

Dominican Republic
1970  
1971  
1973  
1974  
1978  
1979  
1980  
1982  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1996  
1997  
2013  
2014  
2017  

Mexico
1970  
1972  
1973  
1974  
1975  
1976  
1977  
1978  
1979  
1980  
1981  
1982  
1983  
1984  
1985  
1986  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2003  
2004  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Philippines
1970  
1971  
1972  
1974  
1975  
1976  
1977  
1978  
1979  
1980  
1981  
1982  
1983  
1984  
1985  
1986  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Greece
1970  
1971  
1972  
1973  
1974  
1975  
1976  
1977  
1978  
1979  
1980  
1981  
1982  
1983  
1984  
1985  
1986  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1995  


2008  
2009  
2010  
2012  
2013  
2014  
2015  
2016  
2017  

Australia
1970  
1971  
1972  
1974  
1978  
1979  
1980  
1981  
1982  
1986  
1988  
1989  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
2000  
2001  
2006  
2008  
2009  
2010  
2013  
2014  
2015  
2016  
2017  

Pakistan
1970  
1974  
1975  
1976  
1978  
1979  
1980  
1981  
1982  
1983  
1984  
1985  
1986  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Zambia
1971  
1973  
1975  
1977  
1978  
1979  
1981  
1985  
1986  
1987  
1988  
1989  
1990  
1995  
1996  
1999  
2000  
2002  
2017  

Sweden
1971  
1972  
1975  
1978  
1979  
1986  
1983  
1989  
1990  
1991  
1992  
1994  
1995  
1996  
1997  
1999  
2000  
2005  
2006  
2008  
2009  
2010  
2011  
2012  
2014  
2015  
2016  
2017  

Costa Rica
1971  
1976  
1977  
1978  
1979  
1980  
1981  

1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2002  
2007  
2008  
2012  
2013  
2014  
2015  
2016  
2017  

Kenya
1975  
1976  
1978  
1980  
1989  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
2000  
2001  
2002  
2003  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Iraq
1975  
1976  
1979  
1980  
1981  
1982  
1983  
1984  
1987  
1988  
1989  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Somalia
1975  
1980  
1981  
1983  
1984  
1987  
1988  
1989  
1990  
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Tanzania
1975  
1979  
1982  
1994  
1995  
1997  
1998  
2000  
2001  
2002  
2004  
2008  
2010  
2013  
2014  
2015  
2016  
2017  

Sri Lank

2007  
2008  
2009  
2010  
2011  
2013  
2014  
2015  
2016  
2017  

Cuba
1991  
1992  
1994  
1996  
1997  
2007  

Croatia
1991  
1992  
1994  
1995  
1996  
1997  
2000  
2001  
2005  
2008  
2009  
2013  

Georgia
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Azerbaijan
1991  
1992  
1994  
1995  
1996  
1997  
2000  
2001  
2002  
2007  
2008  
2009  
2014  
2016  
2017  

Madagascar
1991  
1992  
1995  
1996  
1997  
2002  
2006  
2009  
2011  
2012  
2013  
2014  
2016  

Lithuania
1991  
1992  
1995  
1996  
1997  

Burundi
1991  
1992  
1994  
1995  
1996  
1997  
1998  
1999  
2000  
2001  
2002  
2003  
2004  
2005  
2006  
2007  
2008  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

Ukraine
1991  
1992  
1994  
1995  
1996  
1997  
1999  
2000  
2002  
2003  
2007  
2008  
2009  
2010  
2011  
2012  
2013  
2014  
2015  
2016  
2017  

"\ngtd[gtd['country_txt'] == 'Philippines'].index\n\ngtd.at[gtd[(gtd['country_txt'] == 'Philippines') & (gtd['iyear'] == 1970)].index, 'gdp'] = gdp[(gdp['Country Name'] == 'Philippines') & (gdp['Year'] == 1970)]['Value'].tolist()[0]\n\ngtd[gtd['country_txt'] == 'Philippines']\n"

In [20]:
# Just checking
gtd[(gtd['country_txt'] == 'Germany') & (gtd['iyear'] == 1972)]

,iyear,imonth,iday,country_txt,extended,country,region,success,attacktype1,targtype1,natlty1,weaptype1,gdp,inf,pop
eventid,,,,,,,,,,,,,,,
197202020001,1972,2,2,Germany,0,362,8,1,3,11,216.0,6,2.986672e+11,4.531857,78688452.0
197202080001,1972,2,8,Germany,0,362,8,1,3,21,75.0,6,2.986672e+11,4.531857,78688452.0
197202220001,1972,2,22,Germany,0,362,8,1,3,21,75.0,6,2.986672e+11,4.531857,78688452.0
197204110001,1972,4,11,Germany,0,362,8,1,2,4,217.0,8,2.986672e+11,4.531857,78688452.0
197204110002,1972,4,11,Germany,0,362,8,0,7,4,217.0,8,2.986672e+11,4.531857,78688452.0
197204230001,1972,4,23,Germany,0,362,8,0,3,4,217.0,6,2.986672e+11,4.531857,78688452.0
197205040002,1972,5,4,Germany,0,362,8,0,3,7,209.0,6,2.986672e+11,4.531857,78688452.0
197205110001,1972,5,11,Germany,0,362,8,1,3,4,217.0,6,2.986672e+11,4.531857,78688452.0
197205120005,1972,5,12,Germany,0,362,8,1,3,3,75.0,6,2.986672e+11,4.531857,78688452.0


In [21]:
# Save csv
gtd.to_csv("economic-terrorism-data.csv")